In [3]:
import gymnasium as gym
import src.environments.news_vendor as nv

In [4]:
# ── 1.  Make the env in HUMAN‑render mode ───────────────────────────────────────
env = gym.make("NewsVendor-v0", render_mode="human")
obs = env.reset()

# Pygame handle (the renderer keeps the pygame module reference internally)
pygame = env.unwrapped._renderer._pg      # note: renderer attribute is “_pg”, not “_pygame”
clock = pygame.time.Clock()

running = True
pending_order = 0.0                       # order you’re composing this turn
high = env.action_space.high[0]           # = config.max_order_quantity

print(
    "News Vendor  —  arrows adjust order (↑ +10, ↓ -10, → +100, ← -100),\n"
    "Digits build an exact number, Backspace deletes,\n"
    "Enter/Space submits the order, r resets, Esc/q quits."
)

# ── 2.  Main event loop ─────────────────────────────────────────────────────────
while running:
    # Draw current state every frame
    env.render()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        elif event.type == pygame.KEYDOWN:
            k = event.key

            # —— Meta controls ——
            if k in (pygame.K_ESCAPE, pygame.K_q):
                running = False
            elif k == pygame.K_r:                 # hard reset
                obs, info = env.reset()
                pending_order = 0.0

            # —— Incremental adjustments ——
            elif k == pygame.K_UP:                # +10
                pending_order += 10
            elif k == pygame.K_DOWN:              # −10
                pending_order -= 10
            elif k == pygame.K_RIGHT:             # +100
                pending_order += 100
            elif k == pygame.K_LEFT:              # −100
                pending_order -= 100

            # —— Build exact quantity with digits ——
            elif pygame.K_0 <= k <= pygame.K_9:
                digit = k - pygame.K_0
                pending_order = pending_order * 10 + digit
            elif k == pygame.K_BACKSPACE:
                pending_order = int(pending_order // 10)

            # —— Submit the order ——
            elif k in (pygame.K_RETURN, pygame.K_SPACE):
                qty = max(0.0, min(pending_order, high))     # clip to action bounds
                obs, reward, terminated, truncated, info = env.step(qty)
                print(f"t={env.unwrapped.current_step:2d}  order={qty:5.0f}  reward={reward:7.2f}")
                pending_order = 0.0

                if terminated or truncated:
                    print("Episode finished – press 'r' to play again.")

    # Keep ‘pending_order’ legal & show it in the title bar
    pending_order = max(0.0, min(pending_order, high))
    pygame.display.set_caption(f"News Vendor — pending order: {int(pending_order)}")

    clock.tick(30)   # keep event loop responsive (30 fps cap)

env.close()

News Vendor  —  arrows adjust order (↑ +10, ↓ -10, → +100, ← -100),
Digits build an exact number, Backspace deletes,
Enter/Space submits the order, r resets, Esc/q quits.
t= 1  order=   20  reward=-725.58
t= 2  order=    0  reward=-798.44
t= 3  order=    0  reward=-798.44
t= 4  order=    0  reward=-919.42
t= 5  order=    0  reward=-854.89
